In [1]:
# ================================================================
# Full pipeline: fundamentals + FCFF/DCF + DDM + Multiples + Risk
# Adds Sector + Buy? (Median Market Cap vs Current Market Cap)
# Robust yfinance usage with tiny throttling + progress heartbeats
# ================================================================

import time
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta
from tqdm import tqdm

# --------- Config ---------
TICKERS = [
    "TXG","MMM","AOS","AAON","ABT","ABBV","ACHC","ACN","AYI","ADBE","ADT","AAP","WMS","AMD","ACM","AES","AMG","AFRM","AFL","AGCO",
    "A","ADC","AGNC","AL","APD","ABNB","AKAM","ALK","ALB","ACI","AA","ARE","ALGN","ALLE","ALGM","LNT","ALSN","ALL","ALLY","ALNY",
    "GOOGL","GOOG","MO","AMZN","AMCR","DOX","AMED","AMTM","AS","AEE","AAL","AEP","AXP","AFG","AMH","AIG","AMT","AWK","COLD","AMP",
    "AME","AMGN","AMKR","APH","ADI","ANGI","AU","NLY","ANSS","AM","AR","AON","APA","APG","APLS","APO","APPF","AAPL","AMAT","APP",
    "ATR","APTV","ARMK","ACGL","ADM","ARES","ANET","AWI","ARW","AJG","ASH","AIZ","AGO","ALAB","ASTS","T","ATI","TEAM","ATO","AUR",
    "ADSK","ADP","AN","AZO","AVB","AVTR","AVY","CAR","AVT","AXTA","AXS","AXON","AZEK","AZTA","BKR","BALL","BAC","OZK","CG","KMX",
    "CCL","CARR","CRI","CVNA","CASY","CAT","CAVA","CBOE","CBRE","CCCS","CDW","CE","CELH","COR","CNC","CNP","CERT","CF","CRL","SCHW",
    "CHTR","CHE","CC","LNG","CVX","CMG","CHH","COIN","CL","COLB","COLM","CMCSA","CMA","FIX","CBSH","CAG","CNXC","CFLT","COP","ED",
    "STZ","CEG","COO","CPRT","CNM","GLW","CPAY","CTVA","CSGP","COST","CTRA","COTY","CPNG","CUZ","CR","CXT","CACC","CRH","CROX",
    "CRWD","CCI","CCK","CSX","CUBE","CMI","CW","CVS","DHI","DHR","DRI","DAR","DDOG","DVA","DAY","DECK","DE","DAL","DELL","XRAY",
    "DVN","DXCM","FANG","DKS","DLR","ECL","EIX","EW","ELAN","ESTC","EA","ESI","ELV","EME","EMR","EHC","ENOV","ENPH","ENTG","ETR",
    "NVST","EOG","EPAM","EPR","EQT","EFX","EQIX","EQH","ELS","EQR","ESAB","WTRG","ESS","EL","ETSY","HHH","HWM","HPQ","HUBB","HUBS",
    "HUM","HBAN","HII","HUN","H","IAC","IBM","IDA","IEX","IDXX","ITW","ILMN","INCY","INFA","IR","INGM","INGR","INSP","PODD","INTC",
    "IBKR","ICE","IFF","IP","IPG","INTU","ISRG","IVZ","INVH","IONS","IPGP","IQV","IRDM","IRM","ITT","JBL","J","JHX","JHG","JAZZ",
    "JBHT","JEF","JNJ","JCI","JLL","JPM","KRMN","KBR","K","KMPR","KVUE","KDP","KEY","KEYS","KRC","KMB","KIM","KMI","KNSL","KEX",
    "KKR","KLAC","KNX","KSS","KHC","KR","KD","LHX","LH","LRCX","LAMR","LW","LSTR","LVS","LSCC","LAZ","LEA","LEG","LDOS","LEN","LULU",
    "LITE","LYFT","LYB","MTB","MTSI","M","MORN","MOS","MS","MSI","MP","COOP","MSA","MSM","MSCI","MUSA","NDAQ","NTRA","NFG","NSA",
    "NCNO","NTAP","NFLX","NBIX","NFE","NYT","NWL","NEU","NEM","NWSA","NWS","NXST","NEE","NKE","NI","NNN","NDSN","NSC","NTRS","NOC",
    "NCLH","NOV","NRG","NU","NUE","NTNX","NVT","NVDA","NVR","ORLY","OXY","OGE","OKTA","ODFL","ORI","OLN","OLLI","OHI","OMC","ONON",
    "ON","OMF","OKE","ONTO","ORCL","OGN","OSK","OTIS","OVV","OC","PCAR","PKG","PLTR","PANW","PK","PH","PSN","PAYX","PAYC","PCTY",
    "PYPL","PEGA","PENN","PAG","PNR","PEN","PEP","PFGC","PR","PRGO","PSA","PEG","PHM","PSTG","PVH","QGEN","QRVO","QCOM","PWR","QS",
    "DGX","QDEL","QXO","RAL","RL","RRC","RJF","RYN","RBA","RBC","O","RDDT","RRX","REG","REGN","RF","RGA","RS","RNR","RGEN","RSG",
    "RMD","RVMD","RVTY","REXR","REYN","RH","RNG","RITM","RIVN","RLI","RHI","HOOD","RBLX","RKT","ROK","ROIV","ROKU","ROL","ROP","ROST",
    "RCL","RGLD","RPRX","RPM","RTX","RBRK","RYAN","R","SPGI","SWKS","SFD","SJM","SW","SNA","SNOW","SOFI","SOLV","SGI","SON","SHC",
    "SO","SCCO","SSB","LUV","SPB","SPR","SPOT","SSNC","STAG","SARO","SWK","SBUX","STWD","STT","STLD","STE","SF","SYK","SMMT","SUI",
    "SMCI","SYF","SNPS","SNV","SYY","TMUS","TTWO","TLN","TPR","TRGP","TGT","SNX","FTI","TDY","TFX","TEM","THC","TDC","TER","TSLA",
    "TTEK","TXN","TPL","TXRH","TXT","TMO","TFSL","UBER","UI","UDR","UGI","PATH","ULTA","RARE","UAA","UA","UNP","UAL","UPS","URI",
    "UTHR","UWMC","UNH","U","OLED","UHS","UNM","USFD","MTN","VLO","VMI","VVV","VEEV","VTR","VLTO","VRSN","VRSK","VZ","VRTX","VRT",
    "VSTS","VFC","VTRS","VICI","VIK","VKTX","VNOM","VIRT","V","VST","VNT","VNO","VOYA","VMC","WPC","WRB","GWW","WAB","WBA","WMT",
    "DIS","WBD","WM","WAT","WSO","W","WFRD","WBS","WEC","WFC","WELL","WEN","WCC","WST","WAL","WDC","WU","WLK","WEX","WY","WHR",
    "WTM","WMB","WSM","WTW","WSC","WING","WTFC","WOLF","WWD","WDAY","WH","WYNN"
]
YEARS = 4
START_DATE = (datetime.today() - timedelta(days=365*4)).strftime("%Y-%m-%d")

# --------- Field Config ---------
income_fields  = ['Total Revenue', 'Operating Income', 'Pretax Income', 'Tax Provision']
cashflow_fields = ['Operating Cash Flow', 'Capital Expenditure', 'Free Cash Flow',
                   'Depreciation And Amortization', 'Stock Based Compensation']
balance_fields = ['Current Assets', 'Current Liabilities', 'Cash And Cash Equivalents']
info_fields    = ['beta', 'marketCap', 'sharesOutstanding', 'sector']

missing_log = []

# ================================================================
# Core Extraction (fundamentals) — with tiny throttle + progress
# ================================================================
def get_financial_data(ticker_symbol):
    try:
        ticker = yf.Ticker(ticker_symbol)

        income_stmt   = ticker.financials.T.head(YEARS)
        cash_flow     = ticker.cashflow.T.head(YEARS)
        balance_sheet = ticker.balance_sheet.T.head(YEARS)
        info          = ticker.info  # heavier call; acceptable with throttle

        # Fill missing columns and log
        for col in income_fields:
            if col not in income_stmt.columns:
                income_stmt[col] = pd.NA
                missing_log.append((ticker_symbol, 'Income Statement', col))
        income_df = income_stmt[income_fields].copy()
        income_df['Ticker'] = ticker_symbol

        for col in cashflow_fields:
            if col not in cash_flow.columns:
                cash_flow[col] = pd.NA
                missing_log.append((ticker_symbol, 'Cash Flow', col))
        cashflow_df = cash_flow[cashflow_fields].copy()
        cashflow_df['Ticker'] = ticker_symbol

        for col in balance_fields:
            if col not in balance_sheet.columns:
                balance_sheet[col] = pd.NA
                missing_log.append((ticker_symbol, 'Balance Sheet', col))
        balance_df = balance_sheet[balance_fields].copy()
        balance_df['Ticker'] = ticker_symbol

        info_clean = {key: info.get(key, None) for key in info_fields}
        info_clean['Ticker'] = ticker_symbol
        info_df = pd.DataFrame([info_clean])

        return income_df, cashflow_df, balance_df, info_df

    except Exception as e:
        print(f"❌ Error with {ticker_symbol}: {e}")
        return None, None, None, None

all_income, all_cashflow, all_balance, all_info = [], [], [], []
for i, t in enumerate(tqdm(TICKERS), 1):
    income, cashflow, balance, info = get_financial_data(t)
    if income is not None:
        all_income.append(income)
        all_cashflow.append(cashflow)
        all_balance.append(balance)
        all_info.append(info)
    if i % 25 == 0:
        print(f"[fundamentals] processed {i}/{len(TICKERS)}")
    time.sleep(0.25)  # gentle throttle to reduce rate-limit risk

df_income  = pd.concat(all_income).reset_index().rename(columns={'index': 'Date'})
df_cashflow= pd.concat(all_cashflow).reset_index().rename(columns={'index': 'Date'})
df_balance = pd.concat(all_balance).reset_index().rename(columns={'index': 'Date'})
df_info    = pd.concat(all_info).reset_index(drop=True)

for df in [df_income, df_cashflow, df_balance]:
    df['Date'] = pd.to_datetime(df['Date'])
    df.round(0)

# ================================================================
# FCFF (history) + Projection
# ================================================================
def compute_historical_fcff_all_tickers(df_income, df_cashflow, df_balance):
    df = df_income.merge(df_cashflow, on=['Date','Ticker']).merge(df_balance, on=['Date','Ticker'])
    df = df.sort_values(by=['Ticker','Date'])

    df['Tax Rate'] = (df['Tax Provision'] / df['Pretax Income']).clip(lower=0, upper=1).fillna(0.21)
    df['NWC']  = df['Current Assets'] - df['Current Liabilities']
    df['ΔNWC'] = df.groupby('Ticker')['NWC'].diff()

    df['FCFF'] = (
        df['Operating Income'] * (1 - df['Tax Rate']) +
        df['Depreciation And Amortization'] -
        df['Capital Expenditure'] -
        df['ΔNWC']
    )

    return df[['Date','Ticker','FCFF','Operating Income','Tax Rate',
               'Depreciation And Amortization','Capital Expenditure','ΔNWC']].round(2)

fcff_df = compute_historical_fcff_all_tickers(df_income, df_cashflow, df_balance)
fcff_df = fcff_df[fcff_df['FCFF'].notna()]

def project_fcff_from_history(fcff_df, projection_years=3):
    projections_fcff = []
    industry_growth_cap = {
        'Technology': 0.09,'Consumer Defensive': 0.06,'Consumer Cyclical': 0.07,'Healthcare': 0.08,
        'Industrials': 0.05,'Financial Services': 0.06,'Energy': 0.04,'Utilities': 0.03,
        'Basic Materials': 0.05,'Real Estate': 0.04,'Communication Services': 0.08
    }
    industry_cache = {}

    def get_industry(ticker):
        if ticker in industry_cache:
            return industry_cache[ticker]
        try:
            info = yf.Ticker(ticker).info
            industry = info.get('sector', 'Unknown')
        except Exception:
            industry = 'Unknown'
        industry_cache[ticker] = industry
        return industry

    for ticker in fcff_df['Ticker'].unique():
        tdf = fcff_df[fcff_df['Ticker'] == ticker].sort_values('Date').tail(3)
        if len(tdf) < 3:
            continue
        fcff_vals = tdf['FCFF'].values
        if fcff_vals[0] == 0 or fcff_vals[1] == 0 or any(pd.isna(fcff_vals)):
            continue
        g1 = (fcff_vals[1] - fcff_vals[0]) / fcff_vals[0]
        g2 = (fcff_vals[2] - fcff_vals[1]) / fcff_vals[1]
        avg_growth = (g1 + g2) / 2
        industry = get_industry(ticker)
        cap = industry_growth_cap.get(industry, 0.06)
        avg_growth = max(min(avg_growth, cap), 0.00)

        last_fcff = fcff_vals[-1]
        last_year = pd.to_datetime(tdf['Date'].max()).year
        for i in range(1, projection_years + 1):
            future_year = last_year + i
            projected_fcff = last_fcff * ((1 + avg_growth) ** i)
            projections_fcff.append({
                'Ticker': ticker,'Year': future_year,'Projected FCFF': round(projected_fcff, 2),
                'Growth Rate': round(avg_growth, 4),'Industry': industry,'Cap Used': cap
            })
    return pd.DataFrame(projections_fcff)

projections_df = project_fcff_from_history(fcff_df, projection_years=3)
proj_summary_df = projections_df.groupby('Ticker').agg(
    Growth_Rate=('Growth Rate','first'),
    Last_Projected_FCFF=('Projected FCFF','last')
).reset_index()

# ================================================================
# CAPM / WACC
# ================================================================
def compute_capm_for_tickers(tickers, risk_free_rate=0.042, market_return=0.09):
    capm_results = []
    for ticker in tickers:
        try:
            beta = yf.Ticker(ticker).info.get('beta', None)
            cost_of_equity = risk_free_rate + beta * (market_return - risk_free_rate) if beta is not None else None
            capm_results.append({'Ticker': ticker,'Beta': round(beta,3) if beta else None,
                                 'Cost of Equity': round(cost_of_equity,4) if cost_of_equity else None})
        except Exception as e:
            capm_results.append({'Ticker': ticker,'Error': str(e)})
        time.sleep(0.02)
    return pd.DataFrame(capm_results)

capm_df = compute_capm_for_tickers(TICKERS)

def compute_cost_of_debt(ticker_symbol, tax_rate=None):
    try:
        ticker = yf.Ticker(ticker_symbol)
        income_stmt = ticker.income_stmt.T
        balance_sheet = ticker.balance_sheet.T
        interest_expense = income_stmt.get('Interest Expense')
        total_debt      = balance_sheet.get('Total Debt')
        if interest_expense is None or total_debt is None:
            return None
        latest_interest = interest_expense.dropna().iloc[-1]
        latest_debt     = total_debt.dropna().iloc[-1]
        if latest_debt == 0:
            return None
        raw_cost = abs(latest_interest) / latest_debt
        return round(raw_cost * (1 - tax_rate), 4) if tax_rate else round(raw_cost, 4)
    except Exception:
        return None

cost_of_debt_df = pd.DataFrame([{'Ticker': t, 'Cost of Debt': compute_cost_of_debt(t, tax_rate=0.21)} for t in TICKERS])

def compute_wacc(ticker_symbol, capm_dict, tax_rate=0.21):
    try:
        ticker = yf.Ticker(ticker_symbol)
        cost_of_equity = capm_dict.get(ticker_symbol)
        cost_of_debt   = compute_cost_of_debt(ticker_symbol, tax_rate=tax_rate)
        equity = ticker.info.get('marketCap', None)
        total_debt_series = ticker.balance_sheet.T.get('Total Debt')
        if equity is None or total_debt_series is None:
            return None
        debt = total_debt_series.dropna().iloc[-1]
        total_value = equity + debt
        if total_value == 0:
            return None
        return round((equity / total_value) * cost_of_equity + (debt / total_value) * cost_of_debt, 4)
    except Exception:
        return None

capm_dict = capm_df.set_index('Ticker')['Cost of Equity'].to_dict()
wacc_df  = pd.DataFrame([{'Ticker': t, 'WACC': compute_wacc(t, capm_dict)} for t in TICKERS])

# ================================================================
# DCF from FCFF projections
# ================================================================
def run_dcf_for_tickers(tickers, fcff_df, projection_years=3, terminal_growth_rate=0.02):
    projections_df = project_fcff_from_history(fcff_df, projection_years=projection_years)
    results = []
    for ticker in tickers:
        wacc_series = wacc_df[wacc_df['Ticker'] == ticker]['WACC']
        wacc = wacc_series.values[0] if not wacc_series.empty else None
        if wacc is None or wacc == 0:
            continue
        proj_rows = projections_df[projections_df['Ticker'] == ticker].sort_values('Year')
        projected_fcffs = proj_rows['Projected FCFF'].values
        if len(projected_fcffs) == 0:
            continue
        discounted_fcffs = [fcff / ((1 + wacc) ** (i + 1)) for i, fcff in enumerate(projected_fcffs)]
        last_fcff = projected_fcffs[-1]
        terminal_value = (last_fcff * (1 + terminal_growth_rate)) / (wacc - terminal_growth_rate)
        discounted_terminal_value = terminal_value / ((1 + wacc) ** len(projected_fcffs))
        dcf_value = sum(discounted_fcffs) + discounted_terminal_value
        results.append({'Ticker': ticker,'DCF Value (B)': round(dcf_value / 1e9, 2)})
    return pd.DataFrame(results)

dcf_df = run_dcf_for_tickers(TICKERS, fcff_df)

# ================================================================
# DDM with sector growth
# ================================================================
def compute_ddm_with_sector_growth(ticker_symbol, sector_growth_dict=None, risk_free_rate=0.042, market_return=0.09):
    try:
        t = yf.Ticker(ticker_symbol)
        info = t.info

        d0 = info.get('dividendRate', None)
        if d0 is None or d0 == 0:
            return {'Ticker': ticker_symbol, 'Error': 'No dividend'}

        beta = info.get('beta', None)
        if beta is None:
            return {'Ticker': ticker_symbol, 'Error': 'No beta'}
        cost_of_equity = risk_free_rate + beta * (market_return - risk_free_rate)

        sector = info.get('sector', 'Unknown')
        if sector_growth_dict is None:
            sector_growth_dict = {
                'Technology': 0.07,'Consumer Defensive': 0.04,'Consumer Cyclical': 0.06,'Healthcare': 0.05,
                'Industrials': 0.04,'Financial Services': 0.05,'Energy': 0.03,'Utilities': 0.03,
                'Basic Materials': 0.04,'Real Estate': 0.03,'Communication Services': 0.05,'Unknown': 0.04
            }
        g = sector_growth_dict.get(sector, 0.04)

        if cost_of_equity <= g:
            return {'Ticker': ticker_symbol, 'Error': f"r <= g (r={cost_of_equity:.2f}, g={g:.2f})"}

        d1 = d0 * (1 + g)
        value_per_share = d1 / (cost_of_equity - g)

        shares_outstanding = info.get('sharesOutstanding', None)
        if shares_outstanding is None:
            return {'Ticker': ticker_symbol, 'Error': 'No shares outstanding'}

        firm_value = value_per_share * shares_outstanding
        firm_value_billions = firm_value / 1e9

        return {
            'Ticker': ticker_symbol,'Sector': sector,'Dividend (D0)': round(d0, 2),
            'Growth Rate': round(g, 4),'Cost of Equity': round(cost_of_equity, 4),
            'Value per Share': round(value_per_share, 2),
            'Shares Outstanding': round(shares_outstanding / 1e9, 2),
            'Intrinsic Firm Value (B)': round(firm_value_billions, 2)
        }
    except Exception as e:
        return {'Ticker': ticker_symbol, 'Error': str(e)}

ddm_results = [compute_ddm_with_sector_growth(t) for t in TICKERS]
df_ddm = pd.DataFrame(ddm_results)

# ================================================================
# Multiples (EV/EBITDA by sector)
# ================================================================
sector_multiples = {
    "Technology": 14.8,"Communication Services": 12.7,"Consumer Defensive": 11.2,"Consumer Cyclical": 12.0,
    "Industrials": 10.1,"Healthcare": 13.3,"Energy": 6.5,"Utilities": 9.0,"Financial Services": 9.8,
    "Real Estate": 10.2,"Basic Materials": 8.7,
}
sector_aliases = {"Consumer Discretionary": "Consumer Cyclical"}

def _get_info(t):
    try:
        return t.get_info() or {}
    except Exception:
        try:
            return t.info or {}
        except Exception:
            return {}

def _safe_get_ebitda(t):
    try:
        inc = t.get_income_stmt(freq="annual")
        if isinstance(inc, pd.DataFrame) and "EBITDA" in inc.index:
            s = pd.to_numeric(inc.loc["EBITDA"].dropna(), errors="coerce")
            if not s.empty and pd.notna(s.iloc[0]):
                return float(s.iloc[0])
    except Exception:
        pass
    info = _get_info(t)
    return info.get("ebitda")

def _safe_get_sector(t):
    info = _get_info(t)
    sector = info.get("sector") or info.get("sectorDisp") or info.get("sectorKey")
    if sector in sector_aliases:
        sector = sector_aliases[sector]
    return sector

def _safe_get_shares_out(t):
    try:
        shares = t.fast_info.get("shares_outstanding")
        if shares:
            return int(shares)
    except Exception:
        pass
    info = _get_info(t)
    return info.get("sharesOutstanding")

def _safe_get_cash_debt(t):
    info = _get_info(t)
    debt = info.get("totalDebt", 0) or 0
    cash = info.get("totalCash", 0) or 0
    return float(debt), float(cash)

def estimate_firm_value_using_multiples(ticker):
    try:
        t = yf.Ticker(ticker)

        ebitda = _safe_get_ebitda(t)
        sector = _safe_get_sector(t)
        shares_out = _safe_get_shares_out(t)
        debt, cash = _safe_get_cash_debt(t)

        if ebitda is None or pd.isna(ebitda) or ebitda == 0:
            return {"Ticker": ticker, "Error": "Missing EBITDA"}
        if not sector:
            return {"Ticker": ticker, "Error": "Missing sector"}
        if sector not in sector_multiples:
            return {"Ticker": ticker, "Sector": sector, "Error": "Sector not in multiples map"}
        if shares_out is None or pd.isna(shares_out) or shares_out == 0:
            return {"Ticker": ticker, "Sector": sector, "Error": "Missing shares outstanding"}

        multiple = sector_multiples[sector]
        enterprise_value = float(ebitda) * float(multiple)
        market_cap = enterprise_value - float(debt) + float(cash)
        value_per_share = market_cap / float(shares_out)

        return {
            "Ticker": ticker,"Sector": sector,"EBITDA": round(ebitda / 1e9, 2),
            "Sector Multiple": multiple,"Enterprise Value (B)": round(enterprise_value / 1e9, 2),
            "Estimated Market Cap (B)": round(market_cap / 1e9, 2),
            "Shares Outstanding (B)": round(shares_out / 1e9, 2),
            "Value per Share ($)": round(value_per_share, 2),"Error": None,
        }
    except Exception as e:
        return {"Ticker": ticker, "Error": str(e)}

df_multiples = pd.DataFrame([estimate_firm_value_using_multiples(t) for t in TICKERS])

# ================================================================
# Prices: batch download once (fewer calls, faster)
# ================================================================
from statsmodels.tsa.arima.model import ARIMA

def compute_risk_metrics(price_df: pd.DataFrame, ticker: str):
    s = price_df[ticker].dropna()
    rets = s.pct_change().dropna()
    if rets.empty:
        return {"Ticker": ticker,"Cumulative Return": None,"Annualized Return": None,
                "Annualized Volatility": None,"Sharpe Ratio": None}
    cumulative_return = float(s.iloc[-1] / s.iloc[0] - 1.0)
    ann_return = float(rets.mean() * 252)
    ann_vol = float(rets.std(ddof=1) * np.sqrt(252))
    sharpe = ann_return / ann_vol if ann_vol != 0 else np.nan
    return {"Ticker": ticker,"Cumulative Return": round(cumulative_return, 4),
            "Annualized Return": round(ann_return, 4),
            "Annualized Volatility": round(ann_vol, 4),
            "Sharpe Ratio": None if np.isnan(sharpe) else round(sharpe, 2)}

def forecast_price_arima(price_series: pd.Series, periods: int = 30):
    series = price_series.dropna().squeeze()
    if len(series) < 60:
        return np.nan
    try:
        model = ARIMA(series, order=(5, 1, 0))
        model_fit = model.fit()
        fc = model_fit.forecast(steps=periods)
        return float(fc.iloc[-1])
    except Exception:
        return np.nan

print("[prices] batch downloading…")
prices = yf.download(
    TICKERS, start=START_DATE, interval="1d",
    auto_adjust=True, progress=False, group_by="column", timeout=60, threads=True
)
# handle different yfinance versions
close = prices["Close"] if isinstance(prices.columns, pd.MultiIndex) and "Close" in prices.columns else (
    prices["Close"] if "Close" in prices.columns else prices.filter(items=TICKERS)
)

rows = []
for i, ticker in enumerate(TICKERS, 1):
    if ticker not in close.columns or close[ticker].dropna().empty:
        rows.append({"Ticker": ticker,"Cumulative Return": None,"Annualized Return": None,
                     "Annualized Volatility": None,"Sharpe Ratio": None,"Predicted Price (30d)": None})
    else:
        s_df = close[[ticker]].rename(columns={ticker: ticker})
        risk = compute_risk_metrics(s_df, ticker)
        pred_price = forecast_price_arima(close[ticker], periods=30)  # comment out for speed if needed
        risk.update({"Predicted Price (30d)": round(pred_price, 2) if pd.notna(pred_price) else None})
        rows.append(risk)
    if i % 50 == 0:
        print(f"[prices] processed {i}/{len(TICKERS)}")

df_results = pd.DataFrame(rows)

# ================================================================
# Combine: DCF + DDM + Multiples -> Median; add current quotes
# ================================================================
dcf_marketcaps        = dcf_df.rename(columns={"DCF Value (B)": "DCF Market Cap (B)"})
ddm_marketcaps        = df_ddm.rename(columns={"Intrinsic Firm Value (B)": "DDM Market Cap (B)"})
multiples_marketcaps  = df_multiples.rename(columns={"Estimated Market Cap (B)": "Multiples Market Cap (B)"})

combined_df = (
    dcf_marketcaps[["Ticker","DCF Market Cap (B)"]]
    .merge(ddm_marketcaps[["Ticker","DDM Market Cap (B)"]], on="Ticker", how="outer")
    .merge(multiples_marketcaps[["Ticker","Multiples Market Cap (B)"]], on="Ticker", how="outer")
)
combined_df["Median Market Cap (B)"] = combined_df[
    ["DCF Market Cap (B)","DDM Market Cap (B)","Multiples Market Cap (B)"]
].median(axis=1, skipna=True)

# Current quotes (with tiny throttle + progress)
current_price_map, current_mcap_map, shares_out_map = {}, {}, {}
for i, tk in enumerate(TICKERS, 1):
    tkr = yf.Ticker(tk)

    price = None
    try:
        price = tkr.fast_info.get("last_price")
    except Exception:
        pass
    if price is None:
        try:
            price = tkr.history(period="1d")["Close"].iloc[-1]
        except Exception:
            price = None
    current_price_map[tk] = price

    mcap = None
    try:
        mcap = tkr.fast_info.get("market_cap")
    except Exception:
        pass
    if mcap is None:
        try:
            mcap = tkr.info.get("marketCap")
        except Exception:
            mcap = None
    current_mcap_map[tk] = (mcap / 1e9) if mcap else None

    shares = None
    try:
        shares = tkr.fast_info.get("shares_outstanding")
    except Exception:
        pass
    if shares is None:
        try:
            shares = tkr.info.get("sharesOutstanding")
        except Exception:
            shares = None
    shares_out_map[tk] = shares

    if i % 50 == 0:
        print(f"[quotes] processed {i}/{len(TICKERS)}")
    time.sleep(0.15)

# Merge quotes into results
df_results = df_results.rename(columns={"Predicted Price (30d)":"Forecast Price (30d)"})
df_results["Current Price"] = pd.to_numeric(df_results["Ticker"].map(current_price_map), errors="coerce")
df_results["Forecast Price (30d)"] = pd.to_numeric(df_results["Forecast Price (30d)"], errors="coerce")

df_results["Forecast Change (%)"] = (
    (df_results["Forecast Price (30d)"] - df_results["Current Price"]) / df_results["Current Price"] * 100
).round(2)

df_results["Current Market Cap (B)"] = df_results["Ticker"].map(current_mcap_map)
df_results["Forecast Market Cap (B)"] = df_results.apply(
    lambda row: (row["Forecast Price (30d)"] * shares_out_map.get(row["Ticker"])) / 1e9
    if pd.notna(row["Forecast Price (30d)"]) and shares_out_map.get(row["Ticker"]) else None,
    axis=1
)

final_combined = combined_df.merge(
    df_results[["Ticker","Current Price","Forecast Price (30d)","Forecast Change (%)",
                "Current Market Cap (B)","Forecast Market Cap (B)"]],
    on="Ticker", how="left"
)

# Sector + Buy? (Median Market Cap vs Current Market Cap)
sector_map = df_info.set_index('Ticker')['sector'].to_dict()
enriched = final_combined.copy()
enriched['Sector'] = enriched['Ticker'].map(sector_map)
enriched['Buy?'] = np.where(
    enriched['Median Market Cap (B)'] > enriched['Current Market Cap (B)'],
    'Yes', 'No'
)

# ================================================================
# Save outputs
# ================================================================
pd.set_option("display.float_format", lambda x: f"{x:,.2f}")

final_df = (
    capm_df
    .merge(cost_of_debt_df, on='Ticker', how='outer')
    .merge(wacc_df, on='Ticker', how='outer')
    .merge(proj_summary_df, on='Ticker', how='outer')
    .merge(dcf_df, on='Ticker', how='outer')
)
final_df.to_csv("final.csv", index=False)
df_ddm.to_csv("df_ddm.csv", index=False)
df_multiples.to_csv("df_multiples.csv", index=False)
df_results.to_csv("df_results.csv", index=False)
final_combined.to_csv("final_combined.csv", index=False)
enriched.to_csv("enriched.csv", index=False)


  4%|██████████▉                                                                                                                                                                                                                                                                             | 24/614 [00:52<20:41,  2.10s/it]

[fundamentals] processed 25/614


  8%|██████████████████████▎                                                                                                                                                                                                                                                                 | 49/614 [01:53<23:33,  2.50s/it]

[fundamentals] processed 50/614


 12%|█████████████████████████████████▋                                                                                                                                                                                                                                                      | 74/614 [02:52<20:59,  2.33s/it]

...

[quotes] processed 50/614


$ANSS: possibly delisted; no price data found  (period=1d)


[quotes] processed 100/614


$AZEK: possibly delisted; no price data found  (period=1d)


[quotes] processed 150/614
[quotes] processed 200/614
[quotes] processed 250/614
[quotes] processed 300/614
[quotes] processed 350/614
[quotes] processed 400/614
[quotes] processed 450/614
[quotes] processed 500/614
[quotes] processed 550/614
[quotes] processed 600/614
